# CVD BB Pullback Strategy Backtest Demo

This notebook demonstrates how to backtest the CVD Bollinger Band Pullback strategy using parquet data files.

In [ ]:
import sys
import os
import polars as pl
import numpy as np
from pathlib import Path

sys.path.append(str(Path(os.getcwd()).parent))
from notebook_utils import setup_notebook_environment, get_data_path

project_root = setup_notebook_environment()

from src.strategies.implementations.cvd_bb_pullback import CVDBBPullbackStrategy
from src.bt_engine.vectorbt_engine import VectorBTEngine

In [2]:
data_path = get_data_path(project_root, "processed", "features", "okx_btc_usdt_perp_1m_2022_10_27-2022_12_27.parquet")
print(f"Loading data from: {data_path}")

data = pl.read_parquet(data_path)

print(f"Data shape: {data.shape}")
print(f"Date range: {data['datetime'].min()} to {data['datetime'].max()}")
print(f"Columns: {list(data.columns)}")
data.head()

Loading data from: c:\Users\marqu\Dropbox\ActiveQuants\quant-strategies\data\processed\features\okx_btc_usdt_perp_1m_2022_10_27-2022_12_27.parquet
Data shape: (87841, 6)
Date range: 2022-10-27 00:00:00+00:00 to 2022-12-27 00:00:00+00:00
Columns: ['datetime', 'open', 'high', 'low', 'close', 'volume']


datetime,open,high,low,close,volume
"datetime[ms, UTC]",f64,f64,f64,f64,f64
2022-10-27 00:00:00 UTC,20765.199219,20765.300781,20742.599609,20748.900391,5872.0
2022-10-27 00:01:00 UTC,20748.900391,20778.199219,20748.800781,20772.900391,4617.0
2022-10-27 00:02:00 UTC,20773.0,20784.699219,20761.5,20780.0,11118.0
2022-10-27 00:03:00 UTC,20776.5,20776.5,20762.800781,20769.199219,2423.0
2022-10-27 00:04:00 UTC,20769.199219,20784.599609,20769.199219,20782.5,5358.0


In [3]:
strategy = CVDBBPullbackStrategy()
engine = VectorBTEngine(
    initial_cash=1000,
    fee_pct=0.05,
    frequency='1min'
)

print(f"Strategy: {strategy.name}")
print(f"Default parameters: {strategy.params}")

Strategy: CVDBBPullbackStrategy
Default parameters: {'bbands_length': 30, 'bbands_stddev': 2.0, 'cvd_length': 50, 'atr_length': 14, 'sl_coef': 2.5, 'tpsl_ratio': 2.0}


In [4]:
param_ranges_small = {
    "bbands_length": np.arange(25, 160, 10),
    "bbands_stddev": np.arange(2.0, 6.0, 0.5),
    "cvd_length": [40],  # np.arange(35, 60, 5),
    "atr_length": [10],  # np.arange(5, 25, 5),
    "sl_coef": [2.0],  # np.arange(2.0, 3.5, 0.5),
    "tpsl_ratio": [2.5],  # np.arange(3.0, 5.5, 0.5)
}

print(f"Parameter combinations: {np.prod([len(v) for v in param_ranges_small.values()])}")

Parameter combinations: 112


In [5]:
results = engine.simulate_portfolios(
    strategy=strategy,
    data=data,
    param_dict=param_ranges_small,
    ticker="BTC-USDT-PERP",
    sizing_method="Risk percent",
    risk_pct=1.0,
    exchange_broker="okx",
    date_range="2021_2024",
    save_results=False,
    indicator_batch_size=50
)

print(f"Backtest completed! Results shape: {results.shape}")

Starting CVDBBPullbackStrategy Backtesting...
Total parameter combinations: 112
Processing in indicator batches of 50


Backtesting Progress:   0%|          | 0/112 [00:00<?, ?it/s]

Backtesting Progress: 100%|██████████| 112/112 [01:44<00:00,  1.07it/s]

All batches processed. Total combinations: 112
Backtest completed! Results shape: (112, 25)


In [6]:
top_results = results.top_k(10, by='sharpe_ratio')

print("Top 10 parameter combinations by Sharpe ratio:")
display(top_results[['CVDBB_bbands_length', 'CVDBB_bbands_stddev', 'CVDBB_cvd_length', 
                    'total_return_pct', 'sharpe_ratio', 'max_drawdown_pct', 
                    'total_trades', 'win_rate_pct']])

Top 10 parameter combinations by Sharpe ratio:


CVDBB_bbands_length,CVDBB_bbands_stddev,CVDBB_cvd_length,total_return_pct,sharpe_ratio,max_drawdown_pct,total_trades,win_rate_pct
i64,f64,i64,f64,f64,f64,i64,f64
145,5.5,40,-39.819421,-10.411527,40.150889,479,30.271399
155,5.5,40,-40.144577,-10.86853,40.50144,456,28.947368
135,5.0,40,-44.225045,-11.340692,44.553563,493,28.600406
155,4.0,40,-50.292281,-11.446861,50.579464,516,29.651163
145,3.5,40,-53.673617,-11.454522,55.035035,562,31.316726
135,4.5,40,-48.136489,-11.805902,49.117781,524,28.816794
155,5.0,40,-45.722611,-11.964529,46.046217,467,28.265525
135,4.0,40,-52.66961,-12.165863,52.947363,540,28.888889
135,5.5,40,-44.161688,-12.317905,44.490578,486,28.1893


In [7]:
import plotly.express as px

fig = px.scatter(
    results, 
    x='max_drawdown_pct', 
    y='total_return_pct',
    color='sharpe_ratio',
    size='total_trades',
    hover_data=['CVDBB_bbands_length', 'CVDBB_cvd_length', 'win_rate_pct'],
    title='Strategy Performance: Return vs Risk'
)
fig.show()